In [ ]:
import pandas as pd

In [ ]:
# Define tickers and date range
tickers = ["TSLA", "BND", "SPY"]
start_date = "2015-07-01"
end_date = "2025-07-31"

# Download data for all tickers
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', auto_adjust=False)

# Separate each dataset
tsla_df = data['TSLA'].copy()
bnd_df = data['BND'].copy()
spy_df = data['SPY'].copy()

# Reset index for clean DataFrame
tsla_df.reset_index(inplace=True)
bnd_df.reset_index(inplace=True)
spy_df.reset_index(inplace=True)

# Optional: Calculate daily volatility (close-to-close returns std dev over a rolling window)
def add_volatility(df, window=21):
    df['Daily Return'] = df['Adj Close'].pct_change()
    df['Volatility'] = df['Daily Return'].rolling(window).std() * (252**0.5)  # Annualized
    return df

tsla_df = add_volatility(tsla_df)
bnd_df = add_volatility(bnd_df)
spy_df = add_volatility(spy_df)

# Save to CSV (if needed)
tsla_df.to_csv("TSLA_data.csv", index=False)
bnd_df.to_csv("BND_data.csv", index=False)
spy_df.to_csv("SPY_data.csv", index=False)

# Display summaries
print("TSLA Sample Data:\n", tsla_df.head())
print("\nBND Sample Data:\n", bnd_df.head())
print("\nSPY Sample Data:\n", spy_df.head())
